In [77]:
import pandas as pd
import pubchempy as pcp
from tqdm import tqdm
from rdkit.Chem import MolFromSmiles

In [39]:
chembl = pd.read_csv('chembl_4_full.csv')[['synonyms', 'pubchem_cid', 'chembl_id', 'inchi_key', 'smiles', 'withdrawn']]
drugbank = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/raw/structure links.csv')[['DrugBank ID', 'Name', 
                                                                                                        'Drug Groups', 'InChIKey', 'SMILES',
                                                                                                       'PubChem Compound ID', 'ChEMBL ID']]
chembl['synonyms'] = chembl['synonyms'].str.lower()
chembl.rename(columns={'withdrawn': 'chembl_withdrawn',
                      'synonyms': 'name'}, inplace=True)

In [40]:
drugbank.rename(columns = {'DrugBank ID': 'drugbank_id',
                          'Name': 'name',
                          'Drug Groups': 'drug_groups',
                          'InChIKey': 'inchi_key',
                          'SMILES': 'smiles',
                          'PubChem Compound ID': 'pubchem_cid',
                          'ChEMBL ID': 'chembl_id'}, inplace=True)

In [41]:
drugbank['drugbank_withdrawn'] = 0
drugbank.loc[drugbank['drug_groups'].str.contains('withdrawn'), 'drugbank_withdrawn'] = 1

In [42]:
chembl['dataset'] = 'chembl'
drugbank['dataset'] = 'drugbank'

In [43]:
drugbank.loc[((drugbank['inchi_key'].isin(chembl['inchi_key'])) | 
            (drugbank['pubchem_cid'].isin(chembl['pubchem_cid'])) |
            (drugbank['chembl_id'].isin(chembl['chembl_id']))), 'dataset'] = 'both'

In [44]:
chembl.loc[((chembl['inchi_key'].isin(drugbank['inchi_key'])) | 
            (chembl['pubchem_cid'].isin(drugbank['pubchem_cid'])) |
            (chembl['chembl_id'].isin(drugbank['chembl_id']))), 'dataset'] = 'both'

In [46]:
chembl['drugbank_withdrawn'] = 'missing'
drugbank['chembl_withdrawn'] = 'missing'

In [47]:
chembl['drugbank_id'] = 'missing'

In [48]:
drugbank.drop(columns='drug_groups', inplace=True)

In [51]:
drugbank

,drugbank_id,name,inchi_key,smiles,pubchem_cid,chembl_id,drugbank_withdrawn,dataset,chembl_withdrawn
0,DB00006,Bivalirudin,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,16129704.0,CHEMBL2103749,0,drugbank,missing
1,DB00007,Leuprolide,GFIJNRVAKGFPGQ-LIJARHBVSA-N,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,657181.0,CHEMBL1201199,0,drugbank,missing
2,DB00014,Goserelin,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,5311128.0,CHEMBL1201247,0,drugbank,missing
3,DB00027,Gramicidin D,NDAYQJDHGXTBJL-MWWSRJDJSA-N,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,45267103.0,CHEMBL557217,0,drugbank,missing
4,DB00035,Desmopressin,NFLWUMRGJYTJIN-PNIOQBSNSA-N,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,5311065.0,CHEMBL1429,0,drugbank,missing
...,...,...,...,...,...,...,...,...,...
2670,DB15934,Ferric glycerophosphate,CBEYZWBXIPZYNH-UHFFFAOYSA-J,[Fe++].[Fe++].OCC(O)COP([O-])([O-])=O.OCCC(CCO...,NaN,NaN,0,drugbank,missing
2671,DB15950,Sodium camphorsulfonate,AWMAOFAHBPCBHJ-YZUKSGEXSA-M,[Na+].[H][C@]12CC[C@@](CS([O-])(=O)=O)(C(=O)C1...,NaN,NaN,0,drugbank,missing
2672,DB15982,Berotralstat,UXNXMBYCBRBRFD-MUUNZHRXSA-N,NCC1=CC(=CC=C1)N1N=C(C=C1C(=O)NC1=CC(=CC=C1F)[...,NaN,NaN,0,drugbank,missing
2673,DB16019,Gallium Ga-68 gozetotide,AEBYHKKMCWUMKX-LNTZDJBBSA-K,[H+].[68Ga+3].OC(=O)CC[C@H](NC(=O)N[C@@H](CCCC...,NaN,NaN,0,drugbank,missing


In [53]:
for i in tqdm(drugbank['inchi_key']):
    try:
        cid = pcp.get_cids(i, 'inchikey', 'compound')[0]
        old_cid = drugbank.loc[drugbank['inchi_key'] == i]['pubchem_cid'].isna().values[0]
        if old_cid == True:
            drugbank.loc[drugbank['inchi_key'] == i, 'pubchem_cid'] = cid
    except:
        print('Molecule not found')

  0%|          | 13/2675 [00:07<23:13,  1.91it/s]

Molecule not found


  3%|▎         | 90/2675 [01:00<40:44,  1.06it/s]

Molecule not found


  4%|▍         | 102/2675 [01:07<23:27,  1.83it/s]

...

 99%|█████████▉| 2643/2675 [24:31<00:17,  1.81it/s]

Molecule not found


100%|█████████▉| 2666/2675 [24:43<00:04,  1.86it/s]

Molecule not found


100%|██████████| 2675/2675 [24:49<00:00,  1.80it/s]


In [60]:
drugbank_in_chembl.loc[drugbank_in_chembl['chembl_id'].isna()]

,drugbank_id,name,inchi_key,smiles,pubchem_cid,chembl_id,drugbank_withdrawn,dataset,chembl_withdrawn
2444,DB13931,Netarsudil,OURRXQUGYQRVML-AREMUKBSSA-N,CC1=CC(C)=C(C=C1)C(=O)OCC1=CC=C(C=C1)[C@@H](CN...,66599893.0,NaN,0,both,missing
2660,DB15685,Selpercatinib,XIIOFHFUYBLOLW-UHFFFAOYSA-N,COC1=NC=C(CN2C3CC2CN(C3)C2=CC=C(C=N2)C2=CC(OCC...,134436906.0,NaN,0,both,missing


In [61]:
drugbank.loc[drugbank['drugbank_id'] == 'DB13931', 'chembl_id'] = "CHEMBL4594250"
drugbank.loc[drugbank['drugbank_id'] == 'DB15685', 'chembl_id'] = "CHEMBL4559134"

In [72]:
drugbank_in_chembl = drugbank.loc[drugbank['dataset'] == 'both']

In [66]:
drugbank_remaining = drugbank.loc[drugbank['dataset'] == 'drugbank']

In [67]:
joined = pd.concat([chembl, drugbank_remaining])

/home/dionizije/anaconda3/envs/withdrawn/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [73]:
for i in drugbank_in_chembl['chembl_id']:
    label = drugbank_in_chembl.loc[drugbank_in_chembl['chembl_id'] == i]['drugbank_withdrawn'].values[0]
    joined.loc[joined['chembl_id'] == i, 'drugbank_withdrawn'] = label

In [75]:
joined = joined[['name', 'pubchem_cid', 'chembl_id', 'drugbank_id', 'inchi_key', 'dataset', 'smiles', 'drugbank_withdrawn', 'chembl_withdrawn']]

In [76]:
joined

,name,pubchem_cid,chembl_id,drugbank_id,inchi_key,dataset,smiles,drugbank_withdrawn,chembl_withdrawn
0,ebrotidine,65869,CHEMBL1742471,missing,ZQHFZHPUZXNPMF-UHFFFAOYSA-N,chembl,N=C(N)Nc1nc(CSCCN/C=N/S(=O)(=O)c2ccc(Br)cc2)cs1,missing,1
1,dihydroxyphosphinecarboxylic acid oxide|foscar...,3415,CHEMBL666,missing,ZJAOAACCNHFJAH-UHFFFAOYSA-N,both,O=C(O)P(=O)(O)O,0,0
2,ci-825|nsc-218321|nipent|pd 81565|pd-81565|pen...,439693,CHEMBL1580,missing,FPVKHBSQESCIEP-JQCXWYLXSA-N,both,OC[C@H]1O[C@@H](n2cnc3c2N=CNC[C@H]3O)C[C@@H]1O,0,0
3,cytotec|misoprostol|mysodelle|sc-29333|topogyne,5282381,CHEMBL606,missing,OJLOPKGSLYJEMD-URPKTTJQSA-N,both,CCCCC(C)(O)C/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1CC...,0,0
4,benznidazole|nsc-299972|ro 07-1051|ro-07-1051|...,31593,CHEMBL110,missing,CULUWZNBISUWAS-UHFFFAOYSA-N,both,O=C(Cn1ccnc1[N+](=O)[O-])NCc1ccccc1,0,0
...,...,...,...,...,...,...,...,...,...
2670,Ferric glycerophosphate,14765,NaN,DB15934,CBEYZWBXIPZYNH-UHFFFAOYSA-J,drugbank,[Fe++].[Fe++].OCC(O)COP([O-])([O-])=O.OCCC(CCO...,0,missing
2671,Sodium camphorsulfonate,5.48401e+06,NaN,DB15950,AWMAOFAHBPCBHJ-YZUKSGEXSA-M,drugbank,[Na+].[H][C@]12CC[C@@](CS([O-])(=O)=O)(C(=O)C1...,0,missing
2672,Berotralstat,1.37528e+08,NaN,DB15982,UXNXMBYCBRBRFD-MUUNZHRXSA-N,drugbank,NCC1=CC(=CC=C1)N1N=C(C=C1C(=O)NC1=CC(=CC=C1F)[...,0,missing
2673,Gallium Ga-68 gozetotide,9.18002e+07,NaN,DB16019,AEBYHKKMCWUMKX-LNTZDJBBSA-K,drugbank,[H+].[68Ga+3].OC(=O)CC[C@H](NC(=O)N[C@@H](CCCC...,0,missing


In [80]:
joined = joined.loc[~joined['smiles'].isna()]

In [83]:
bad_valence = []
for i in joined['smiles']:
    try:
        mol = MolFromSmiles(i)
        for atom in mol.GetAtoms():
            atom.GetFormalCharge()
    except AttributeError:
        bad_valence.append(i)
    

RDKit ERROR: [11:56:57] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [11:56:57] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [11:56:57] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [11:56:57] Explicit valence for atom # 84 N, 4, is greater than permitted
RDKit ERROR: [11:56:57] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
RDKit ERROR: [11:56:57] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'
RDKit ERROR: [11:56:57] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit WARNING: [11:56:57] WARNING: not

In [85]:
joined = joined.loc[~joined['smiles'].isin(bad_valence)]

In [89]:
joined.drop_duplicates(subset='pubchem_cid', inplace=True)
joined.drop_duplicates(subset='inchi_key', inplace=True)
joined.drop_duplicates(subset='chembl_id', inplace=True)
joined.drop_duplicates(subset='smiles', inplace=True)

In [92]:
joined.to_csv('/home/dionizije/Documents/drug_attrition_oracle/data/chembl_drugbank_p4_21_7.csv')

In [140]:
joined['withdrawn_chembl_+'] = joined['chembl_withdrawn']
joined['withdrawn_drugbank_+'] = joined['drugbank_withdrawn']

In [133]:
chembl_withdrawn = joined.loc[joined['chembl_withdrawn'] == 1]['chembl_id'].values
drugbank_withdrawn = joined.loc[joined['drugbank_withdrawn'] == 1]['chembl_id'].values

In [119]:
joined

,name,pubchem_cid,chembl_id,drugbank_id,inchi_key,dataset,smiles,drugbank_withdrawn,chembl_withdrawn,withdrawn_chembl_+,withdrawn_drugbank_+
0,ebrotidine,65869,CHEMBL1742471,missing,ZQHFZHPUZXNPMF-UHFFFAOYSA-N,chembl,N=C(N)Nc1nc(CSCCN/C=N/S(=O)(=O)c2ccc(Br)cc2)cs1,missing,1,1,missing
1,dihydroxyphosphinecarboxylic acid oxide|foscar...,3415,CHEMBL666,missing,ZJAOAACCNHFJAH-UHFFFAOYSA-N,both,O=C(O)P(=O)(O)O,0,0,0,0
2,ci-825|nsc-218321|nipent|pd 81565|pd-81565|pen...,439693,CHEMBL1580,missing,FPVKHBSQESCIEP-JQCXWYLXSA-N,both,OC[C@H]1O[C@@H](n2cnc3c2N=CNC[C@H]3O)C[C@@H]1O,0,0,0,0
3,cytotec|misoprostol|mysodelle|sc-29333|topogyne,5282381,CHEMBL606,missing,OJLOPKGSLYJEMD-URPKTTJQSA-N,both,CCCCC(C)(O)C/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1CC...,0,0,0,0
4,benznidazole|nsc-299972|ro 07-1051|ro-07-1051|...,31593,CHEMBL110,missing,CULUWZNBISUWAS-UHFFFAOYSA-N,both,O=C(Cn1ccnc1[N+](=O)[O-])NCc1ccccc1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2663,Mebrofenin,54158,CHEMBL1605443,DB15779,MHPZZZZLAQGTHT-UHFFFAOYSA-N,drugbank,CC1=CC(C)=C(Br)C(C)=C1NC(=O)CN(CC(O)=O)CC(O)=O,0,missing,missing,0
2667,Acetylene,6326,CHEMBL116336,DB15906,HSFWRNGVRCDJHI-UHFFFAOYSA-N,drugbank,C#C,0,missing,missing,0
2668,Anethole,637563,CHEMBL452630,DB15916,RUVINXPYWBROJD-ONEGZZNKSA-N,drugbank,COC1=CC=C(\C=C\C)C=C1,0,missing,missing,0
2669,Ammonium carbonate,10480,CHEMBL2105849,DB15926,PRKQVKDSMLBJBJ-UHFFFAOYSA-N,drugbank,[NH4+].[NH4+].[O-]C([O-])=O,0,missing,missing,0


In [149]:
if name in drugbank_withdrawn:
    print(True)

In [151]:
for index, row in joined.iterrows():
    name = row['chembl_id']
    if row['withdrawn_chembl_+'] == 'missing':
        if name in drugbank_withdrawn:
            joined.loc[joined['chembl_id'] == name, 'withdrawn_chembl_+'] = 1
        else:
            joined.loc[joined['chembl_id'] == name, 'withdrawn_chembl_+'] = 0
    if row['withdrawn_drugbank_+'] == 'missing':
        if name in chembl_withdrawn:
            joined.loc[joined['chembl_id'] == name, 'withdrawn_drugbank_+'] = 1
        else:
            joined.loc[joined['chembl_id'] == name, 'withdrawn_drugbank_+'] = 0

In [154]:
j

,name,pubchem_cid,chembl_id,drugbank_id,inchi_key,dataset,smiles,drugbank_withdrawn,chembl_withdrawn,withdrawn_chembl_+,withdrawn_drugbank_+
0,ebrotidine,65869,CHEMBL1742471,missing,ZQHFZHPUZXNPMF-UHFFFAOYSA-N,chembl,N=C(N)Nc1nc(CSCCN/C=N/S(=O)(=O)c2ccc(Br)cc2)cs1,missing,1,1,1
1,dihydroxyphosphinecarboxylic acid oxide|foscar...,3415,CHEMBL666,missing,ZJAOAACCNHFJAH-UHFFFAOYSA-N,both,O=C(O)P(=O)(O)O,0,0,0,0
2,ci-825|nsc-218321|nipent|pd 81565|pd-81565|pen...,439693,CHEMBL1580,missing,FPVKHBSQESCIEP-JQCXWYLXSA-N,both,OC[C@H]1O[C@@H](n2cnc3c2N=CNC[C@H]3O)C[C@@H]1O,0,0,0,0
3,cytotec|misoprostol|mysodelle|sc-29333|topogyne,5282381,CHEMBL606,missing,OJLOPKGSLYJEMD-URPKTTJQSA-N,both,CCCCC(C)(O)C/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1CC...,0,0,0,0
4,benznidazole|nsc-299972|ro 07-1051|ro-07-1051|...,31593,CHEMBL110,missing,CULUWZNBISUWAS-UHFFFAOYSA-N,both,O=C(Cn1ccnc1[N+](=O)[O-])NCc1ccccc1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2663,Mebrofenin,54158,CHEMBL1605443,DB15779,MHPZZZZLAQGTHT-UHFFFAOYSA-N,drugbank,CC1=CC(C)=C(Br)C(C)=C1NC(=O)CN(CC(O)=O)CC(O)=O,0,missing,0,0
2667,Acetylene,6326,CHEMBL116336,DB15906,HSFWRNGVRCDJHI-UHFFFAOYSA-N,drugbank,C#C,0,missing,0,0
2668,Anethole,637563,CHEMBL452630,DB15916,RUVINXPYWBROJD-ONEGZZNKSA-N,drugbank,COC1=CC=C(\C=C\C)C=C1,0,missing,0,0
2669,Ammonium carbonate,10480,CHEMBL2105849,DB15926,PRKQVKDSMLBJBJ-UHFFFAOYSA-N,drugbank,[NH4+].[NH4+].[O-]C([O-])=O,0,missing,0,0
